# Video Analytics (Jupyter Version - Video Path Input & Output)

This notebook:
- Takes a video from a local path
- Runs YOLO detection
- Tracks people
- Generates alerts
- Saves processed output video

## Step 1: Install Dependencies

In [1]:
!pip install ultralytics opencv-python matplotlib numpy

## Step 2: Import Libraries

In [2]:
import cv2
import numpy as np
from ultralytics import YOLO

## Step 3: Provide Video Path

In [3]:
# ENTER YOUR VIDEO PATH HERE
video_path = 'input_video.mp4'
output_path = 'output_processed_video.mp4'

## Step 4: Load YOLO Model

In [4]:
model = YOLO('yolov8n.pt')

## Step 5: Initialize Video Reader & Writer

In [5]:
cap = cv2.VideoCapture(video_path)

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

heatmap = None
object_time = {}
danger_zone = (200, 200, 400, 400)

## Step 6: Process Video Frame-by-Frame

In [6]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    if heatmap is None:
        heatmap = np.zeros((frame.shape[0], frame.shape[1]), dtype=np.float32)

    results = model(frame)

    for r in results:
        boxes = r.boxes.xyxy.cpu().numpy()
        classes = r.boxes.cls.cpu().numpy()

        for box, cls in zip(boxes, classes):
            x1, y1, x2, y2 = map(int, box)

            if int(cls) == 0:
                cx = int((x1+x2)/2)
                cy = int((y1+y2)/2)

                heatmap[cy, cx] += 1

                key = (cx, cy)
                object_time[key] = object_time.get(key, 0) + 1

                if object_time[key] > 50:
                    cv2.putText(frame, 'Loitering', (x1,y1), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)

                dx1, dy1, dx2, dy2 = danger_zone
                if dx1 < cx < dx2 and dy1 < cy < dy2:
                    cv2.putText(frame, 'Danger Zone', (x1,y2), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,255), 2)

                cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),2)

    cv2.rectangle(frame,(danger_zone[0],danger_zone[1]),(danger_zone[2],danger_zone[3]),(0,0,255),2)

    out.write(frame)

cap.release()
out.release()
print('Processing complete. Output saved at:', output_path)


0: 384x640 10 persons, 1 handbag, 64.4ms
Speed: 98.9ms preprocess, 64.4ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 1 handbag, 43.3ms
Speed: 3.3ms preprocess, 43.3ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 1 handbag, 56.0ms
Speed: 2.8ms preprocess, 56.0ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 1 handbag, 55.0ms
Speed: 4.0ms preprocess, 55.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 1 handbag, 54.9ms
Speed: 2.4ms preprocess, 54.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 1 handbag, 61.2ms
Speed: 2.6ms preprocess, 61.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 1 handbag, 78.5ms
Speed: 4.0ms preprocess, 78.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 10 persons, 1 